In [156]:
import json
import glob
from IPython.display import clear_output

Start by importing the data and splitting it by lines

In [169]:
filename = "tehstone_s3.json"
#filename = "season3.json"
#filename = "blaseball-log-season3.json"
#filename = "combined-blaseball-log-1596691883485.json"

In [170]:
raw = []
with open(filename, 'r') as f:
    raw += f.read().split("\n")
if(raw[-1] == ""):
    raw = raw[:-1]

In [171]:
json_lines = []
for line in raw:
    json_lines.append(json.loads(line))

How many lines did we get?

In [172]:
print(len(json_lines))

52883


In [173]:
def legalStateChange(old_state, new_state):
    # Ignore bad starting game states
    if(old_state["lastUpdate"] == 'Game over.' and new_state["lastUpdate"] != 'Game over.'):
        return None
    if(old_state["lastUpdate"] == ""):
        return None
    
    # Check for skipped innings
    if(new_state["inning"] - old_state["inning"] > 1 or new_state["inning"] - old_state["inning"] < 0):
        return (0, f"Skipped an inning. {old_state['inning']} -> {new_state['inning']}")
    
    # Check for skipped half innings
    if(new_state["inning"] != old_state["inning"] and new_state["topOfInning"] == old_state["topOfInning"]):
        return (1, f"Skipped a half inning. {(old_state['inning'], old_state['topOfInning'])} -> " + 
                f"{(new_state['inning'], new_state['topOfInning'])}")
    if(new_state["inning"] == old_state["inning"] and new_state["topOfInning"] and not old_state["topOfInning"]):
        return (1, f"Skipped a half inning. {(old_state['inning'], old_state['topOfInning'])} -> " + 
                f"{(new_state['inning'], new_state['topOfInning'])}")
    
    # Check for score decrements GRAND UNSLAM
    if(new_state["awayScore"] < old_state["awayScore"]):
        return (2, f"awayScore went down. {old_state['awayScore']} -> {new_state['awayScore']}")
    if(new_state["homeScore"] < old_state["homeScore"]):
        return (2, f"homeScore went down. {old_state['homeScore']} -> {new_state['homeScore']}")
    
    # Check for too many scores
    if(new_state["awayScore"] - old_state["awayScore"] > len(old_state["basesOccupied"]) + 1):
        return (3, f"awayScore incremented by too many runs." +
                f"{old_state['awayScore']} -> {new_state['awayScore']} w/bases: {old_state['basesOccupied']}")
    if(new_state["homeScore"] - old_state["homeScore"] > len(old_state["basesOccupied"]) + 1):
        return (3, f"homeScore incremented by too many runs." +
                f"{old_state['homeScore']} -> {new_state['homeScore']} w/bases: {old_state['basesOccupied']}")
    
    # Check for skips in balls/strikes
    if(new_state["atBatBalls"] - old_state["atBatBalls"] > 1):
        return (4, f"skipped a ball: {old_state['atBatBalls']} -> {new_state['atBatBalls']}")
    
    if(new_state["atBatStrikes"] - old_state["atBatStrikes"] > 1):
        return (4, f"skipped a strike: {old_state['atBatStrikes']} -> {new_state['atBatStrikes']}")

    # Check for skips in player roster
    if(abs(new_state["awayTeamBatterCount"] - old_state["awayTeamBatterCount"]) > 1):
        return (5, f"awayTeamBatterCount skipped a batter." +
                f"{old_state['awayTeamBatterCount']} -> {new_state['awayTeamBatterCount']}")
    if(abs(new_state["homeTeamBatterCount"] - old_state["homeTeamBatterCount"]) > 1):
        return (5, f"homeTeamBatterCount skipped a batter." +
                f"{old_state['homeTeamBatterCount']} -> {new_state['homeTeamBatterCount']}")
    return None

In [174]:
def updateUniqueID(update):
    return (update["inning"], update["topOfInning"], update["atBatBalls"], update["atBatStrikes"], update["lastUpdate"])

In [175]:
updates_by_game = {}
count = 0
errors = []
for line in json_lines:
    for update in line["schedule"]:
        if update["_id"] not in updates_by_game:
            updates_by_game[update["_id"]] = [update]
            continue;
        
        last_update = updates_by_game[update["_id"]][-1]
        if(last_update == update):
            continue
        
        change = legalStateChange(last_update, update)
        if(change):
            count += 1
            # print(update["_id"], ":", change)
            errors.append((update["_id"], update["inning"], change[0], change[1], updateUniqueID(update)))
        updates_by_game[update["_id"]].append(update)

In [176]:
len(updates_by_game)

1002

In [177]:
count

859

In [151]:
def inspectGameError(error):
    updates = updates_by_game[error[0]]
    print("ERROR WAS", error[3])
    for update in updates:
        if(updateUniqueID(update) == error[4]):
            print("E => ", end='')
        else:
            print("     ", end='')
        if(error[2] == 0):
            print(f"Inning: {update['inning']}, {update['lastUpdate']}")
        elif(abs(update["inning"] - error[1]) < 2):
            if(error[2] == 1):
                print(f"Inning/isTop: {(update['inning'], update['topOfInning'])}, {update['lastUpdate']}")
            elif(error[2] == 2):
                print(f"(awayScore, homeScore): {(update['awayScore'], update['homeScore'])}, {update['lastUpdate']}")
            elif(error[2] == 3):
                print(f"Bases/(awayScore, homeScore): {(update['basesOccupied'], (update['awayScore'], update['homeScore']))}, {update['lastUpdate']}")
            elif(error[2] == 4):
                print(f"(balls, strikes): {(update['atBatBalls'], update['atBatStrikes'])}, {update['lastUpdate']}")
            elif(error[2] == 5):
                print(update['lastUpdate'])

In [158]:
trueErrors = []
for error in errors:
    clear_output(wait=True)
    inspectGameError(error)
    haveInput = False
    while(not haveInput):
        result = input("Press 'y' for a true error and 'n' for a false error: ")
        if(result[0] == 'y'):
            trueErrors.append(error)
            haveInput = True
        elif(result[0] == 'n'):
            haveInput = True

ERROR WAS skipped a ball: 0 -> 2
     (balls, strikes): (0, 0), 
     (balls, strikes): (0, 0), Play ball!
     (balls, strikes): (0, 0), Top of 1, Chicago Firefighters batting.
     (balls, strikes): (0, 0), Paula Mason batting for the Firefighters.
     (balls, strikes): (0, 0), Paula Mason hit a ground out to Taiga Quitter.
     (balls, strikes): (0, 0), Declan Suzanne batting for the Firefighters.
     (balls, strikes): (0, 0), Declan Suzanne hit a ground out to Wanda Pothos.
     (balls, strikes): (0, 0), Joshua Butt batting for the Firefighters.
     (balls, strikes): (1, 0), Ball. 1-0
     (balls, strikes): (2, 0), Ball. 2-0
     (balls, strikes): (2, 1), Strike, looking. 2-1
     (balls, strikes): (3, 1), Ball. 3-1
     (balls, strikes): (3, 2), Strike, swinging. 3-2
     (balls, strikes): (0, 0), Joshua Butt strikes out looking.
     (balls, strikes): (0, 0), Bottom of 1, Los Angeles Tacos batting.
     (balls, strikes): (0, 0), Basilio Preston batting for the Tacos.
     (bal

KeyboardInterrupt: Interrupted by user